In [ ]:
import tensorflow as tf
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [ ]:
!pip install tensorflow-datasets

In [ ]:
import tensorflow_datasets as tfds

tfds.list_builders()

['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'aloha_mobile',
 'amazon_us_reviews',
 'anli',
 'answer_equivalence',
 'arc',
 'asqa',
 'asset',
 'assin2',
 'asu_table_top_converted_externally_to_rlds',
 'austin_buds_dataset_converted_externally_to_rlds',
 'austin_sailor_dataset_converted_externally_to_rlds',
 'austin_sirius_dataset_converted_externally_to_rlds',
 'bair_robot_pushing_small',
 'bc_z',
 'bccd',
 'beans',
 'bee_dataset',
 'beir',
 'berkeley_autolab_ur5',
 'berkeley_cable_routing',
 'berkeley_fanuc_manipulation',
 'berkeley_gnm_cory_hall',
 'berkeley_gnm_recon',
 'berkeley_gnm_sac_son',
 'berkeley_mvp_converted_externally_to_rlds',
 'berkeley_rpt_converted_externally_to_rlds',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'ble_wind_field',
 'blimp',
 'booksum',
 'bool_q',
 'bot_adversarial_dialogue',
 'bridge',
 'bridge_data_msr',
 'bucc',
 'c4',
 'c4_wsrs',
 'caltech10

In [ ]:
cifar_builder = tfds.builder("cifar100")
cifar_builder.download_and_prepare()

print(cifar_builder.info)

tfds.core.DatasetInfo(
    name='cifar100',
    full_name='cifar100/3.0.2',
    description="""
    This dataset is just like the CIFAR-10, except it has 100 classes containing 600 images each. There are 500 training images and 100 testing images per class. The 100 classes in the CIFAR-100 are grouped into 20 superclasses. Each image comes with a "fine" label (the class to which it belongs) and a "coarse" label (the superclass to which it belongs).
    """,
    homepage='https://www.cs.toronto.edu/~kriz/cifar.html',
    data_dir='/root/tensorflow_datasets/cifar100/3.0.2',
    file_format=tfrecord,
    download_size=160.71 MiB,
    dataset_size=132.03 MiB,
    features=FeaturesDict({
        'coarse_label': ClassLabel(shape=(), dtype=int64, num_classes=20),
        'id': Text(shape=(), dtype=string),
        'image': Image(shape=(32, 32, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=100),
    }),
    supervised_keys=('image', 'label'),
    disable_shuf

In [ ]:
print(cifar_builder.info.features["label"].names)

['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree',

In [ ]:
print(cifar_builder.info.features["coarse_label"].names)

['aquatic_mammals', 'fish', 'flowers', 'food_containers', 'fruit_and_vegetables', 'household_electrical_devices', 'household_furniture', 'insects', 'large_carnivores', 'large_man-made_outdoor_things', 'large_natural_outdoor_scenes', 'large_omnivores_and_herbivores', 'medium_mammals', 'non-insect_invertebrates', 'people', 'reptiles', 'small_mammals', 'trees', 'vehicles_1', 'vehicles_2']


In [ ]:
import math

input_shape = [224, 224, 3]
batch_size = 32
num_epochs = 1

train_cifar_dataset = cifar_builder.as_dataset(split=tfds.Split.TRAIN)
val_cifar_dataset = cifar_builder.as_dataset(split=tfds.Split.TEST)
num_classes = cifar_builder.info.features["label"].num_classes

num_train_imgs = cifar_builder.info.splits['train'].num_examples
num_val_imgs = cifar_builder.info.splits['test'].num_examples

In [ ]:
print('Training dataset instance: {}'.format(train_cifar_dataset))

Training dataset instance: <_PrefetchDataset element_spec={'coarse_label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'id': TensorSpec(shape=(), dtype=tf.string, name=None), 'image': TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>


In [ ]:
train_cifar_dataset = train_cifar_dataset.repeat(num_epochs).shuffle(10000)

In [ ]:
def _prepare_data_fn(features, input_shape, augment = False):
  input_shape = tf.convert_to_tensor(input_shape)
  image = features['image']
  label = features['label']
  image = tf.image.convert_image_dtype(image, tf.float32)
  if augment:
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
    image = tf.clip_by_value(image, 0.0, 1.0)
    random_scale_factor = tf.random.uniform([1], minval=1., maxval=1.4, dtype=tf.float32)
    scaled_height = tf.cast(tf.cast(input_shape[0], tf.float32)*random_scale_factor, tf.int32)
    scaled_width = tf.cast(tf.cast(input_shape[1], tf.float32)*random_scale_factor, tf.int32)
    scaled_shape = tf.squeeze(tf.stack([scaled_height, scaled_width]))
    image = tf.image.resize(image, scaled_shape)
  else:
    image = tf.image.resize(image, input_shape[:2])
  return image, label



In [ ]:

import functools

prepare_data_fn_for_train = functools.partial(_prepare_data_fn,
                                              input_shape=input_shape,
                                              augment=True)

train_cifar_dataset = train_cifar_dataset.map(prepare_data_fn_for_train, num_parallel_calls=4)

# We also ask the dataset to batch the samples:
train_cifar_dataset = train_cifar_dataset.batch(batch_size)

train_cifar_dataset = train_cifar_dataset.prefetch(1)

In [ ]:
prepare_data_fn_for_val = functools.partial(_prepare_data_fn,
                                              input_shape=input_shape,
                                              augment=False)

val_cifar_dataset = (val_cifar_dataset
                     .repeat()
                     .map(prepare_data_fn_for_val, num_parallel_calls=4)
                     .batch(batch_size)
                     .prefetch(1))

In [ ]:
train_steps_per_epoch = math.ceil(num_train_imgs / batch_size)
val_steps_per_epoch   = math.ceil(num_val_imgs / batch_size)

In [1]:
def _res_conv(filters, kernel_size=3, padding='same', strides=1, use_relu=True, use_bias=False, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(1e-4)):
  def layer_fn(x):
    conv = Conv2D(
        filters = filters,
        kernel_size = kernel_size,
        padding = padding,
        strides = strides,
        use_bias = use_bias,
        kernel_initializer = kernel_initializer,
        kernel_regularizer = kernel_regularizer,
        name = name + '_c'
    )(x)
    res = BatchNormalization(axis = -1, name = name + '_bn')(conv)
    if use_relu:
      res = Activation("relu", name = name + '_r')(res)
    return res
  return layer_fn

NameError: name 'regularizers' is not defined

In [ ]:
def merge_with_shortcut(kernel_initializer='he_normal', kernel_regualizer=regalurizer.l2(1e-4)):
  def layer_fn(x, x_residual):
    x_shape = tf.keras.backend.int_shape(x)
    x_residual_shape = tf.keras.backend.int_shape(x_residual)
    if x_shape == x_residual_shape:
      shortcut = x
    else:
      strides = (
          int(round(x_shape[1] / x_residual_shape[1])),
          int(round(x_shape[2] / x_residual_shape[2]))
      )
      x_residual_channels = x_residual_shape[3]
      shortcut = Conv2D(
          filters = x_residual_channels,
          kernel_size = (1, 1),
          padding = "valid",
          strides = strides,
          kernel_initializer = kernel_initializer,
          kernel_regularizer = kernel_regularizer,
          name = name + _shortcut_c'
      )(x)
      merge = add([shortcut, x_residual])
      return merge
    return layer_fn

In [ ]:
def _residual_block_basic(filters, kernel_size=3, strides=1, use_bias=False, name = 'res_basic', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(1e-4)):
  def layer_fn(x):
    x_conv1 = _res_conv(filters, kernel_size=kernel_size, padding='same', strides = strides, use_relu=True, use_bias=use_bias, kernel_initializer=kernel_initializer, kernel_regulrizer=kernel_regularizer, name=name + '_cbr_1')(x)
    x_residual = _res_conv(filters, kernel_size=kernel_size, padding='same', strides = 1, use_relu=False, use_bias=use_bias, kernel_initializer=kernel_initializer, kernel_regulrizer=kernel_regularizer, name=name + '_cbr_1')(x_conv1)
    merge = merge_with_shortcut(kernel_initializer, kernel_regularizer, name = name)(x, x_residual)
    merge = Activation('relu')(merge)
    return merge
  return layer_fn

In [ ]:
def _residual_block_bottleneck(filters, kernel_size=3, strides=1, use_bias = False, name = 'res_bottleneck', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(1e-4)):
  def layer_fn(x):
    x_bottleneck = _res_conv(filters = filters, kernel_size=kernel_size, padding='valid', strides = strides, use_relu = True, use_bias = use_bias, kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer, name = name + '_cbr1')(x)
    x_conv = _res_conv(filters = filters, kernel_size=kernel_size, padding='same', strides = 1, use_relu = True, use_bias = use_bias, kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer, name = name + '_cbr2')(x_bottleneck)
    x_residual = _res_conv(filters = filters*4, kernel_size=1, padding='valid', strides=1, use_relu =False, use_bias = use_bias, kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer, name = name + '_cbr3')(x_conv)
    merge = merge_with_shortcut(kernel_initializer, kernel_regularizer, name=name)(x, x_residual)
    merge = Activation('relu')(merge)
    return merge
  return layer_fn

In [ ]:
def _residual_macroblock(block_fn, repetitions=3, kernel_size=3, strides_1st_block=1, use_bias = False, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(1e-4), name = 'res_macroblock'):
  def layer_fn(x):
    for i in range(repetitions):
      block_name = "{}_{}".format(name, i)
      strides = strides_1st_block if i == 0 else 1
      x = block_fn(filters=filters, kernel_size=kernel_size, strides = strides, use_bias = use_bias, kernel_initializer=kernel_initialzer, kernel_regularizer=kernel_regularizer, name=block_name)(x)
      return x
    return layer_fn

In [ ]:
def Resnet(input_shape, num_classes= 1000, block_fn = _residual_block_basic, repetitions=(2, 2, 2, 2), use_bias = False, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(1e-4)):
  inputs = Input(shape=input_shape)
  conv = _res_conv(filters=64, kernel_size=7, strides=2, use_relu=True, use_bias=use_bias, kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regulaarizer)(inputs)
  maxpool = MaxPooling2D(pool_size=3, strides=2, padding='same')(conv)

  filters = 64
  strides = 2
  res_block = maxpool
  for i, repet in enumerate(repetitions):
    block_strides = strides if i != 0 else 1
    macroblock_name = "block_{}".format(i)
    res_block = _residual_macroblock(
        block_fn = block_fn, repetitions = repet, name = macroblock_name, filters=filters, strides_1st_block=block_strides, use_bias = use_bias, kernel_intializer=kernel_initializer, kernel_regularizer=kernel_regularizer
    )(res_block)
    filters = min(filters*2, 1024)

  res_spatial_dim = tf.keras.backend.int_shape(res_block)[1:3]
  avg_pool = AveragePooling2D(pool_size=res_spatial_dim)(res_block)
  flatten = Flatten()(avg_pool)
  predictions = Dense(units=num_classes, kernel_intializer=kernel_initializer, activation='softmax')(flatten)
  model = Model(inputs=inputs, outputs=predictions)
  return model

In [ ]:
def ResNet18(input_shape, num_classes=1000, use_bias=True,
              kernel_initializer='he_normal', kernel_regularizer=None):
    return ResNet(input_shape, num_classes, block_fn=_residual_block_basic, repetitions=(2, 2, 2, 2),
                  use_bias=use_bias, kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)


def ResNet34(input_shape, num_classes=1000, use_bias=True,
             kernel_initializer='he_normal', kernel_regularizer=None):
    return ResNet(input_shape, num_classes, block_fn=_residual_block_basic, repetitions=(3, 4, 6, 3),
                  use_bias=use_bias, kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)


def ResNet50(input_shape, num_classes=1000, use_bias=True,
             kernel_initializer='he_normal', kernel_regularizer=None):
    # Note: ResNet50 is similar to ResNet34,
    # with the basic blocks replaced by bottleneck ones (3 conv layers each instead of 2)
    return ResNet(input_shape, num_classes, block_fn=_residual_block_bottleneck, repetitions=(3, 4, 6, 3),
                  use_bias=use_bias, kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)


def ResNet101(input_shape, num_classes=1000, use_bias=True,
             kernel_initializer='he_normal', kernel_regularizer=None):
    return ResNet(input_shape, num_classes, block_fn=_residual_block_bottleneck, repetitions=(3, 4, 23, 3),
                  use_bias=use_bias, kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)


def ResNet152(input_shape, num_classes=1000, use_bias=True,
             kernel_initializer='he_normal', kernel_regularizer=None):
    return ResNet(input_shape, num_classes, block_fn=_residual_block_bottleneck, repetitions=(3, 8, 36, 3),
                  use_bias=use_bias, kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)